In [999]:
import numpy as np  
import pandas as pd
from pandas import Series, DataFrame
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.python.ops import confusion_matrix
from tensorflow.python.ops import math_ops

import matplotlib.pyplot as plt

In [1000]:
#Data=pd.read_csv('C:/Users/dkoch/Documents/tensorflow/R_Input.csv')
Data=pd.read_csv('R_Input.csv')
Data=Data.fillna(0)

Data['dprio']=Data['dprio']/(1*np.mean(Data['dprio']))

Data['dprio0']=Data['dprio']**0
Data['dprio2']=Data['dprio']**2
Data['dprio3']=Data['dprio']**3
Data=Data.rename(columns={'dprio': 'dprio1'})
Data['pax_total']=Data['pax_total']-Data['group_pax']
Data['pax_total']=Data['pax_total'].apply(lambda x: x if x>0 else 0)

direction=pd.get_dummies(Data['direction'],drop_first=True)
direction.columns=['direction:'+x for x in direction.columns]

whichholiday_left_LUX=pd.get_dummies(Data['whichholiday_left_LUX'],drop_first=True)
whichholiday_left_LUX.columns=['whichholiday_left_LUX:'+x for x in whichholiday_left_LUX.columns]

whichholiday_right_LUX=pd.get_dummies(Data['whichholiday_right_LUX'],drop_first=True)
whichholiday_right_LUX.columns=['whichholiday_right_LUX:'+x for x in whichholiday_right_LUX.columns]

Data=pd.concat([Data,direction,whichholiday_left_LUX,whichholiday_right_LUX],axis=1)

Data.index=Data['calendar_date']+'_'+Data['direction']

Data['weekday']=Data['calendar_date'].apply(lambda x: pd.to_datetime(x).weekday()+1)
Data['yearweek']=Data['calendar_date'].apply(lambda x: pd.to_datetime(x).isocalendar()[1])

dprio=Data[['dprio'+str(i) for i in range(0,4)]].drop_duplicates().transpose()
dprio.columns=range(365,-1,-1)
dprio=dprio[[x for x  in range(0,366)]]

X_data=Data.drop_duplicates(subset=['calendar_date','direction'])[[x for x in Data.columns if 'direction:' in x]\
+['weekday','yearweek']\
+[x for x in Data.columns if 'whichholiday_left_LUX:' in x]\
+[x for x in Data.columns if 'whichholiday_right_LUX:' in x]\
+['holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']]

X_data=X_data[['direction:O','weekday','yearweek',
'holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']]

Data['concat']=Data['calendar_date']+'_'+Data['direction']
y_data=Data[['concat','dprio1','pax_total']].pivot(index='concat',columns='dprio1',values='pax_total')

X_train=X_data.loc[[x for x in X_data.index if '2019' not in x],:]
y_train=y_data.loc[[x for x in y_data.index if '2019' not in x],:]

X_test=X_data.loc[[x for x in X_data.index if '2019' in x],:]

n_input=X_train.shape[1]
n_hidden_1=n_input*10
n_hidden_2=n_input*100
n_output=y_train.shape[1]

In [1001]:
#dprio.loc['dprio1',:]=(dprio.loc['dprio1',:]-np.mean(dprio.loc['dprio1',:]))/np.std(dprio.loc['dprio1',:])
#dprio.loc['dprio1',:]=-dprio.loc['dprio1',:]

In [1002]:
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=1)

X_train=X_train.values
y_train=y_train.values
X_test=X_test.values

In [1003]:
# Parameters
learning_rate = 10
training_epochs = 100

X = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_output])

keep_prob = tf.placeholder(tf.float32)

# # Dictionary of Weights and Biases
weights = {
   'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1],mean=0,stddev=0.001)),
   'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],mean=0,stddev=0.001)),
   'out': tf.Variable(tf.random_normal([n_hidden_2, n_output],mean=0,stddev=0.001))
}

biases = {
   'b1': tf.Variable(tf.random_normal([n_hidden_1],mean=0,stddev=0.001)),
   'b2': tf.Variable(tf.random_normal([n_hidden_2],mean=0,stddev=0.001)),
   #'out': tf.Variable(tf.random_normal([n_output],mean=0,stddev=0.001))
}

# Model Forwar§d Propagation step
def forward_propagation(x):
    # Hidden layer1
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.sigmoid(layer_1)
    # Hidden layer2
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    #layer_2 = tf.nn.leaky_relu(layer_2)
    # Output fully connected layer
    out_layer=tf.matmul(layer_2, weights['out'])
    out_layer = tf.exp(out_layer)
    return out_layer


# Model Outputs
yhat = forward_propagation(X)

In [1004]:
lasso_param = tf.constant(0.00000000099)
penal_w1=tf.multiply(tf.reduce_sum(tf.math.abs(weights['w1'])),lasso_param)
penal_w2=tf.multiply(tf.reduce_sum(tf.math.abs(weights['w2'])),lasso_param)
penal_wout=tf.multiply(tf.reduce_sum(tf.math.abs(weights['out'])),lasso_param)
penal_b1=tf.multiply(tf.reduce_sum(tf.math.abs(biases['b1'])),lasso_param)
penal_b2=tf.multiply(tf.reduce_sum(tf.math.abs(biases['b2'])),lasso_param)
#penal_bout=tf.multiply(tf.reduce_sum(tf.math.abs(biases['out'])),lasso_param)
#heavyside_step = tf.truediv(1., tf.add(1., tf.exp(tf.multiply(-50., tf.subtract(X, lasso_param)))))
#regularization_param = tf.multiply(heavyside_step, 99.)

#cost = tf.reduce_mean(tf.abs(yhat-y))

error = tf.subtract(yhat, y)
cost = tf.reduce_mean(tf.maximum(0.15*error, -0.85*error),axis=0)

#cost=tf.losses.absolute_difference(yhat,y)
#cost = tf.add(tf.add(tf.add(tf.add(tf.add(tf.reduce_mean(tf.square(yhat - y)), penal_w1), penal_w2), penal_wout),penal_b1), penal_b2)
#train = tf.train.GradientDescentOptimizer(1).minimize(cost)
train = tf.train.AdadeltaOptimizer(0.1).minimize(cost)

In [1005]:
saver = tf.train.Saver(max_to_keep=1)

# launch a session to run our graph defined above. 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        # get a sample of the dataset and run the optimizer, which calculates a forward pass and then runs the backpropagation algorithm to improve the weights
        train.run(feed_dict = {X: X_train, y: y_train})
        # every 100 iterations, print out the accuracy
        # accuracy and loss are both functions that take (x, y) pairs as input, and run a forward pass through the network to obtain a prediction, and then compares the prediction with the actual y.
        loss = cost.eval(feed_dict = {X: X_train, y: y_train})
        pred = sess.run(yhat, feed_dict={X:X_test})
        if i%100==0:
            print("Epoch: {}, loss: {}".format(i, loss))
            print(pred)
    saver.save(sess, "mymodel")
#     builder.add_meta_graph_and_variables(sess,
#                                        [tf.saved_model.tag_constants.TRAINING],
#                                        signature_def_map=None,
#                                        assets_collection=None)
# builder.save()

Epoch: 0, loss: [0.22415937 0.31578568 0.28184214 0.26703387 0.29203936 0.30991694
 0.23240185 0.269695   0.36592114 0.36424464 0.29944107 0.35251287
 0.27667817 0.24371086 0.32821247 0.9472     0.35795376 0.30935904
 0.4699635  0.37011304 0.28309655 0.39970863 0.4406348  0.4578065
 0.35181543 0.603063   0.45808905 0.24594195 0.5181264  0.49888188
 0.5389368  0.45126343 0.6149269  0.60738635 0.3146672  0.6586356
 0.57149917 0.59607744 0.53882194 0.85335606 0.6212199  0.3603391
 0.6185872  0.6469199  0.65763533 0.45642138 0.59035337 0.5837952
 0.3272353  0.58491063 0.5871447  0.5477414  0.4159221  0.65653104
 0.61309767 0.3324023  0.5308474  0.5114339  0.5397983  0.4763968
 0.58015233 0.42611873 0.31103307 0.44426936 0.5282048  0.52946275
 0.45754856 0.5505494  0.53784245 0.27877292 0.5382643  0.52080953
 0.59273887 0.5384096  0.5062785  0.51325893 0.33729085 0.6227649
 0.51061076 0.54607993 0.47094795 0.524284   0.45991692 0.31004977
 0.5569745  0.5397967  0.5178774  0.46857208 0.48170

Epoch: 200, loss: [0.19762538 0.31157762 0.26666668 0.22405405 0.28794625 0.3100602
 0.16718264 0.2703239  0.35565084 0.34892902 0.29948926 0.347857
 0.2641649  0.19799551 0.3282124  0.9033826  0.33961543 0.3095801
 0.41175726 0.36003736 0.25945354 0.36999524 0.37726858 0.39610857
 0.34940237 0.5058959  0.4196187  0.21671768 0.47244686 0.43930885
 0.46064344 0.4456981  0.51014495 0.530984   0.30182347 0.5645633
 0.49873266 0.52241623 0.506106   0.77851653 0.5172507  0.3611899
 0.51399696 0.52983797 0.5058875  0.42557216 0.48457512 0.51094705
 0.3247987  0.48212194 0.51825917 0.46568376 0.4062373  0.5189611
 0.5360858  0.3276114  0.47132793 0.45216808 0.48564744 0.4524277
 0.5110817  0.4080241  0.30574423 0.411317   0.4794257  0.48739898
 0.4420417  0.47101673 0.49098897 0.2522348  0.5043367  0.47946432
 0.5438858  0.49361578 0.46790445 0.46286583 0.31633762 0.55183065
 0.46252063 0.5036431  0.45647818 0.44868222 0.41692463 0.28084612
 0.48347405 0.49087593 0.4867538  0.45303962 0.43600

Epoch: 400, loss: [1.97535425e-01 3.11418176e-01 2.66247272e-01 2.23862126e-01
 2.87985861e-01 3.09931248e-01 1.66692302e-01 2.70932913e-01
 3.53519827e-01 3.51331234e-01 2.99850285e-01 3.44892710e-01
 2.63798028e-01 1.97445959e-01 3.23794097e-01 8.97466302e-01
 3.33827645e-01 3.09476346e-01 4.03934896e-01 3.61445516e-01
 2.59165108e-01 3.67849737e-01 3.72055233e-01 3.98930848e-01
 3.43788177e-01 5.06750166e-01 4.13925558e-01 2.16349810e-01
 4.48023438e-01 4.34304386e-01 4.50054407e-01 4.40779746e-01
 5.04960537e-01 5.21704137e-01 3.01723152e-01 5.58278918e-01
 4.93865788e-01 5.26943147e-01 4.94114399e-01 7.78456390e-01
 5.20622730e-01 3.60978931e-01 5.07187188e-01 5.23772061e-01
 5.09494185e-01 4.17840153e-01 4.77178007e-01 5.05561769e-01
 3.24481040e-01 4.81271446e-01 5.18942893e-01 4.60812330e-01
 3.95189762e-01 5.11895895e-01 5.34103513e-01 3.27651352e-01
 4.53371286e-01 4.47808892e-01 4.71896887e-01 4.41607803e-01
 4.82256800e-01 3.98707867e-01 3.05630893e-01 4.08703208e-01
 4.621

Epoch: 600, loss: [1.97327659e-01 3.11271042e-01 2.66076177e-01 2.23496869e-01
 2.87648767e-01 3.09863269e-01 1.66417539e-01 2.70578146e-01
 3.53844941e-01 3.52599382e-01 2.99666286e-01 3.43442082e-01
 2.63664216e-01 1.97242841e-01 3.22695881e-01 9.00871933e-01
 3.34773958e-01 3.09455186e-01 4.04905468e-01 3.61976206e-01
 2.58990675e-01 3.67364854e-01 3.74659419e-01 3.95674765e-01
 3.41569662e-01 5.08103073e-01 4.12403435e-01 2.16164127e-01
 4.37655181e-01 4.33519423e-01 4.47556406e-01 4.37696546e-01
 5.04916191e-01 5.22249043e-01 3.01617980e-01 5.55830002e-01
 4.91250485e-01 5.22933602e-01 4.85466033e-01 7.75233626e-01
 5.21381795e-01 3.60626131e-01 5.07360518e-01 5.23864746e-01
 5.09950519e-01 4.13896412e-01 4.75084692e-01 5.06641567e-01
 3.24676394e-01 4.79732186e-01 5.16794562e-01 4.60907549e-01
 3.90760958e-01 5.10561824e-01 5.32383442e-01 3.27641457e-01
 4.43802804e-01 4.44124967e-01 4.68153328e-01 4.36185300e-01
 4.79298651e-01 3.98570240e-01 3.05492878e-01 4.08850342e-01
 4.547

Epoch: 800, loss: [1.97273195e-01 3.11175436e-01 2.66018152e-01 2.23414630e-01
 2.87529141e-01 2.93195635e-01 1.66347265e-01 2.60290891e-01
 3.53291035e-01 3.50891173e-01 2.99485058e-01 3.43017757e-01
 2.63578087e-01 1.97202355e-01 3.21926862e-01 9.04096484e-01
 3.37534100e-01 3.08637500e-01 4.02146608e-01 3.63879204e-01
 2.58950800e-01 3.61691058e-01 3.77121001e-01 3.97821009e-01
 3.40416342e-01 5.11630714e-01 4.12066698e-01 2.16128752e-01
 4.33496088e-01 4.28028762e-01 4.46369022e-01 4.21238482e-01
 5.08589447e-01 5.25230527e-01 3.01631749e-01 5.55929363e-01
 4.90556777e-01 5.23285568e-01 4.78386641e-01 7.79059112e-01
 5.23613393e-01 3.60399216e-01 5.07996976e-01 5.24620891e-01
 5.08826673e-01 4.08695847e-01 4.76332039e-01 5.03879726e-01
 3.24547976e-01 4.79004413e-01 5.18049002e-01 4.60734367e-01
 3.83170784e-01 5.10106862e-01 5.29964864e-01 3.27735871e-01
 4.40347940e-01 4.39867377e-01 4.65183020e-01 4.23723817e-01
 4.72141623e-01 3.95578295e-01 3.05068642e-01 4.09433901e-01
 4.517

...

Epoch: 9200, loss: [1.97079286e-01 2.90732056e-01 2.27103636e-01 2.23195255e-01
 2.70634353e-01 2.49893188e-01 1.49013653e-01 2.44670019e-01
 3.18172365e-01 3.21632683e-01 2.38583058e-01 2.99753994e-01
 2.41042644e-01 1.90854743e-01 2.87914276e-01 8.63219678e-01
 3.02663803e-01 2.34408438e-01 3.74194533e-01 3.06194276e-01
 2.15468869e-01 3.51208210e-01 3.44907850e-01 3.73481274e-01
 2.85855621e-01 4.50798631e-01 3.81628066e-01 2.16080442e-01
 4.11201000e-01 4.17257428e-01 3.84311497e-01 3.33683550e-01
 4.43636000e-01 4.80906010e-01 3.01555842e-01 5.08155286e-01
 4.84452575e-01 4.73475873e-01 3.89649719e-01 7.50262141e-01
 4.90641326e-01 3.00660968e-01 4.81680334e-01 5.01593411e-01
 4.64307547e-01 3.34652871e-01 4.61920887e-01 4.64600027e-01
 2.62913674e-01 4.44370508e-01 4.88962322e-01 4.46442217e-01
 2.78381020e-01 4.79744166e-01 4.90935206e-01 3.27587903e-01
 4.20653433e-01 4.14436519e-01 4.16985363e-01 3.12019050e-01
 4.45066988e-01 3.62142205e-01 2.48497009e-01 3.99108887e-01
 4.06

Epoch: 9400, loss: [1.97078869e-01 2.90677279e-01 2.26828426e-01 2.23194882e-01
 2.70660967e-01 2.49452412e-01 1.48882389e-01 2.44583473e-01
 3.17462742e-01 3.20773274e-01 2.38203853e-01 2.99367607e-01
 2.40926921e-01 1.90663859e-01 2.86663383e-01 8.61107826e-01
 3.00492495e-01 2.33994067e-01 3.73229057e-01 3.05452257e-01
 2.14529172e-01 3.51049066e-01 3.44158322e-01 3.72608364e-01
 2.84592360e-01 4.48667943e-01 3.80706757e-01 2.16079727e-01
 4.10571933e-01 4.16832238e-01 3.83510053e-01 3.32370222e-01
 4.40918267e-01 4.79253590e-01 3.01555127e-01 5.07853150e-01
 4.83758122e-01 4.71014261e-01 3.88841629e-01 7.49493420e-01
 4.89940614e-01 3.00168812e-01 4.81482089e-01 4.99874771e-01
 4.63177353e-01 3.34181875e-01 4.61598396e-01 4.64140266e-01
 2.62499392e-01 4.44006979e-01 4.88512337e-01 4.45614725e-01
 2.77489722e-01 4.79317874e-01 4.90440756e-01 3.27471673e-01
 4.19501752e-01 4.13827002e-01 4.16688830e-01 3.10034543e-01
 4.44771796e-01 3.61106664e-01 2.47891903e-01 3.98313284e-01
 4.06

Epoch: 9600, loss: [1.97078571e-01 2.90606380e-01 2.26699874e-01 2.23194525e-01
 2.70572096e-01 2.49317378e-01 1.48802042e-01 2.44332582e-01
 3.16723138e-01 3.20115030e-01 2.38076791e-01 2.99143404e-01
 2.40787402e-01 1.90515295e-01 2.86453128e-01 8.60471010e-01
 2.99489975e-01 2.33404830e-01 3.72446597e-01 3.04861248e-01
 2.14135513e-01 3.50998193e-01 3.43411207e-01 3.71871769e-01
 2.83290893e-01 4.46999758e-01 3.80404770e-01 2.16078967e-01
 4.10031796e-01 4.16446954e-01 3.83205771e-01 3.31649750e-01
 4.38977540e-01 4.77341861e-01 3.01554501e-01 5.06969631e-01
 4.82905567e-01 4.71043080e-01 3.88443470e-01 7.48785257e-01
 4.89858836e-01 2.99931705e-01 4.81141359e-01 4.99520212e-01
 4.62374002e-01 3.33829790e-01 4.61453944e-01 4.63877261e-01
 2.62340873e-01 4.43704963e-01 4.88188922e-01 4.45313513e-01
 2.77127951e-01 4.79239494e-01 4.90172476e-01 3.15969825e-01
 4.19029832e-01 4.13309097e-01 4.15857881e-01 3.09255511e-01
 4.44595814e-01 3.61008465e-01 2.47730657e-01 3.98178101e-01
 4.05

Epoch: 9800, loss: [1.97078392e-01 2.90568948e-01 2.26555109e-01 2.23194316e-01
 2.70486385e-01 2.49055281e-01 1.48751065e-01 2.44222403e-01
 3.16155016e-01 3.19515437e-01 2.37775177e-01 2.98430383e-01
 2.40674689e-01 1.90421954e-01 2.86538988e-01 8.59982967e-01
 2.98771828e-01 2.33346403e-01 3.71945649e-01 3.04421246e-01
 2.13775188e-01 3.51171911e-01 3.42797011e-01 3.71138155e-01
 2.81906515e-01 4.45365876e-01 3.79930377e-01 2.16078311e-01
 4.09597337e-01 4.16015625e-01 3.83330137e-01 3.30888748e-01
 4.37826395e-01 4.74671334e-01 3.01553756e-01 5.06445110e-01
 4.81863767e-01 4.70802814e-01 3.88076007e-01 7.48292148e-01
 4.89302278e-01 3.00021708e-01 4.81120408e-01 4.99523789e-01
 4.61483181e-01 3.33856553e-01 4.61385250e-01 4.63634551e-01
 2.62416124e-01 4.43432480e-01 4.87868786e-01 4.44474339e-01
 2.76793808e-01 4.79419678e-01 4.89689052e-01 3.07527453e-01
 4.18624699e-01 4.13094550e-01 4.15248692e-01 3.09175223e-01
 4.44446385e-01 3.60722035e-01 2.47055158e-01 3.98009986e-01
 4.05

KeyboardInterrupt: 

In [ ]:
plt.plot(pred[1])

In [ ]:
plt.plot(np.cumsum(pred[44][[x for x in range(365,-1,-1)]]))

In [ ]:
check_sum=[]
for i in range(len(pred)):
    check_sum.append(np.sum(pred[i]))

In [ ]:
oos_index=list([x for x in Data.index if '2019' in x])

In [ ]:
oos_index_unique=[]
for i in oos_index:
    if i not in oos_index_unique:
        oos_index_unique.append(i)

In [ ]:
plt.plot(np.cumsum(Data.loc[oos_index_unique[25],'pax_total'].tolist()))

In [ ]:
np.cumsum(Data.loc[oos_in§dex[40],'pax_total'].tolist())

In [1014]:
138/8

17.25

In [1027]:
Data.groupby('dprio1')['pax_total'].std(0.999)

dprio1
0.000000    0.745544
0.005479    1.070533
0.010959    1.053480
0.016438    0.829746
0.021918    0.939715
0.027397    1.009890
0.032877    0.711763
0.038356    0.895339
0.043836    1.182039
0.049315    1.226596
0.054795    1.023424
0.060274    1.096452
0.065753    1.002722
0.071233    1.095326
0.076712    1.161029
0.082192    8.242775
0.087671    1.210744
0.093151    0.989348
0.098630    1.290148
0.104110    1.426262
0.109589    0.931358
0.115068    1.290417
0.120548    1.520176
0.126027    1.406923
0.131507    1.205138
0.136986    1.777675
0.142466    1.558659
0.147945    0.933841
0.153425    1.448690
0.158904    1.686194
              ...   
1.841096    0.667850
1.846575    0.473077
1.852055    0.685444
1.857534    0.454922
1.863014    0.597961
1.868493    0.456256
1.873973    0.250956
1.879452    0.649093
1.884932    0.461200
1.890411    0.583141
1.895890    0.403145
1.901370    0.467014
1.906849    0.588831
1.912329    0.134382
1.917808    0.664060
1.923288    0.000000
1.9287